## Capstone Project - Winonsin Breast Cancer Diagnosis Deep Learning Revisited






#### Import necessary libraries

In [39]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.model_selection import ShuffleSplit

# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Pretty display for notebooks
%matplotlib inline

#### Load in WBCD dataset

In [40]:
# Load the Boston housing dataset
headers = ["ID","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses","Diagnosis"]
data = pd.read_csv('breast-cancer-wisconsin.csv', names = headers)
data = data.reset_index(drop=True)

#### Handle missing data

In [41]:
data = data.replace('?', np.nan)
data = data.fillna(0)

#### Define a data generation function with random noise

In [42]:
def generateData(data):
    # make a copy of data
    data2 = data

    diagnosis2 = data2['Diagnosis']
    features2 = data2.drop(['ID','Diagnosis'], axis = 1)
    features2_headers = ["CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses"]

    mu, sigma = 0, 0.1 
    # creating a noise with the same dimension as the dataset 
    noise = np.random.normal(mu, sigma, features2.shape) 

    features2 = features2.apply(pd.to_numeric, errors='ignore')

    features2_with_noise = features2.add(pd.DataFrame(noise, columns = features2_headers), fill_value=0)

    data2 = pd.concat([features2_with_noise, pd.DataFrame(diagnosis2)], axis = 1)
    
    return data2

#### Add the new generated dataset into the existing dataset

In [43]:
new_data = generateData(data)
data = data.append(new_data, ignore_index=True)

new_data = generateData(data)
data = data.append(new_data, ignore_index=True)

print("data size = ", data.size)

('data size = ', 30756)


#### Scale dataset to the range of [0, 1]

In [44]:
# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler()
numerical = ["Diagnosis","CT","UCSize","UCShape","MA","SECSize","BN","BC","NN","Mitoses"]
data[numerical] = scaler.fit_transform(data[numerical])

#### Separate Labels/Classes from Features

In [45]:
diagnosis = data['Diagnosis']
features = data.drop(['ID','Diagnosis'], axis = 1)

#### Split dataset into training and testing datasets

In [46]:
from sklearn.model_selection import train_test_split

# Shuffle and split the data into training and testing subsets
X_train, X_test, y_train, y_test = train_test_split(features, diagnosis, test_size=0.25, random_state=42)

#### Reindex 

In [47]:
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

#### Convert Pandas DataFrame to Numpy ndarray

In [48]:
X_train = X_train.values
y_train = y_train.values
X_test  = X_test.values
y_test  = y_test.values

#### Classify dataset using Random Forest Classifier

In [49]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(max_depth=2, random_state=0)
rfc.fit(X_train, y_train)
# y_predict = rfc.predict(X_test)

score = rfc.score(X_test, y_test)
print("score = ", score)

('score = ', 0.96995708154506433)


#### Deep learning

In [50]:
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
import keras
import keras.utils
from keras import utils as np_utils

#### Define NN Architecture

In [51]:
model = Sequential()

#Define your architecture.
model.add(Dense(9, activation='relu', input_dim=9))
model.add(Dropout(0.5))
model.add(Dense(5, activation='relu', input_shape=(9,)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='relu', input_shape=(5,)))
model.add(Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 9)                 90        
_________________________________________________________________
dropout_5 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 50        
_________________________________________________________________
dropout_6 (Dropout)          (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 6         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 2         
Total params: 148
Trainable params: 148
Non-trainable params: 0
_________________________________________________________________


#### Compile NN Model

In [52]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#### Train NN Model

In [53]:
model.fit(X_train, y_train, epochs=100, batch_size=15) # tried epochs = 20, batch_size = 10

Epoch 1/100
2097/2097 [==============================] - 0s - loss: 0.6489 - acc: 0.6872     
Epoch 2/100
2097/2097 [==============================] - 0s - loss: 0.6130 - acc: 0.7601     
Epoch 3/100
2097/2097 [==============================] - 0s - loss: 0.5707 - acc: 0.7907     
Epoch 4/100
2097/2097 [==============================] - 0s - loss: 0.5390 - acc: 0.7997     
Epoch 5/100
2097/2097 [==============================] - 0s - loss: 0.5155 - acc: 0.8240     
Epoch 6/100
2097/2097 [==============================] - 0s - loss: 0.4932 - acc: 0.8336     
Epoch 7/100
2097/2097 [==============================] - 0s - loss: 0.4771 - acc: 0.8336     
Epoch 8/100
2097/2097 [==============================] - 0s - loss: 0.4425 - acc: 0.8588     
Epoch 9/100
2097/2097 [==============================] - 0s - loss: 0.4386 - acc: 0.8498     
Epoch 10/100
2097/2097 [==============================] - 0s - loss: 0.4440 - acc: 0.8379     
Epoch 11/100
2097/2097 [==============================] - 0

2097/2097 [==============================] - 0s - loss: 0.2380 - acc: 0.9270     
Epoch 88/100
2097/2097 [==============================] - 0s - loss: 0.2237 - acc: 0.9313     
Epoch 89/100
2097/2097 [==============================] - 0s - loss: 0.2196 - acc: 0.9380     
Epoch 90/100
2097/2097 [==============================] - 0s - loss: 0.2324 - acc: 0.9304     
Epoch 91/100
2097/2097 [==============================] - 0s - loss: 0.2298 - acc: 0.9266     
Epoch 92/100
2097/2097 [==============================] - 0s - loss: 0.2286 - acc: 0.9285     
Epoch 93/100
2097/2097 [==============================] - 0s - loss: 0.2382 - acc: 0.9299     
Epoch 94/100
2097/2097 [==============================] - 0s - loss: 0.2441 - acc: 0.9242     
Epoch 95/100
2097/2097 [==============================] - 0s - loss: 0.2294 - acc: 0.9309     
Epoch 96/100
2097/2097 [==============================] - 0s - loss: 0.2280 - acc: 0.9242     
Epoch 97/100
2097/2097 [==============================] - 0s - 

#### Test NN Model

In [54]:
score = model.evaluate(X_test, y_test, batch_size=10)
print("score = ", score)

 10/699 [..............................] - ETA: 2s('score = ', [0.12331208833713132, 0.97854076826862346])
